In [0]:
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT REPLACE(SPLIT(current_user(), '@')[0], '.', '_');

USE IDENTIFIER("fhir_workshop." || schema_use);

SELECT
  bundle_uuid
  ,fhir:resourceType::string as bundle_resourceType
  ,fhir:type::string as bundle_type
  ,entry.*
FROM
  fhir_bronze_variant
  ,LATERAL variant_explode(fhir:entry) as entry
where fhir is not null;

SELECT
  bundle_uuid
  ,entry.value:fullUrl::string as fullUrl -- the fullUrl is the primary key of the resource in the bundle, it is only unique inside a bundle and not guaranteed unique across bundles
  ,entry.value:request
  ,entry.value:resource.resourceType::string as resourceType
  ,entry.value:resource
FROM
  fhir_bronze_variant
  ,LATERAL variant_explode(fhir:entry) as entry
where fhir is not null;

with resources as (
  SELECT DISTINCT 
  bundle_uuid || entry.value:fullUrl::string as resource_sk
  ,entry.value:resource.resourceType::string as resourceType
FROM
  fhir_bronze_variant
  ,LATERAL variant_explode(fhir:entry) as entry
where fhir is not null
)
select 
  resourceType
  ,count(resource_sk) as count
from resources
group by resourceType
ORDER BY count desc;


Databricks visualization. Run in Databricks to view.